# Synchronization of ontology files

## Setup

In [2]:
import logging
import os
import sys

# set up module paths for imports
module_path = os.path.abspath(os.path.join('..'))
hercules_sync_path = os.path.abspath(os.path.join('..', 'hercules_sync'))
sys.path.append(module_path)
sys.path.append(hercules_sync_path)

# start logging system and set logging level
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting logger")

INFO:root:Starting logger


## Introduction

In [3]:
source_content = """
#################################################################
# Example ontology.                                             #
# This file is used to test the CI and synchronization systems. #
#################################################################

@prefix ex: <http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:AdministrativePersonnel rdf:type owl:Class ;
                           rdfs:subClassOf  ex:HumanResource ;
                           owl:disjointWith ex:ResearchPersonnel ,
                                            <http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#Research&TeachingPersonnel> .

ex:HumanResource rdf:type owl:Class .

ex:ResearchPersonnel rdf:type owl:Class ;
                     rdfs:subClassOf ex:HumanResource .
"""

target_content = """
#################################################################
# Example ontology.                                             #
# This file is used to test the CI and synchronization systems. #
#################################################################

@prefix ex: <http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix asio: <http://www.asio.es/asioontologies/asio#> .


ex:AdministrativePersonnel rdf:type owl:Class ;
                           rdfs:subClassOf  ex:HumanResource ;
                           owl:disjointWith ex:ChangedPersonnel ,
                                            <http://www.semanticweb.org/spitxa/ontologies/2020/2/asio-human-resource#Research&TeachingPersonnel> .

ex:HumanResource rdf:type owl:Class .

ex:ChangedPersonnel rdf:type owl:Class .

asio:TechnicalPersonnel rdf:type owl:Class ;
                        rdfs:subClassOf asio:HumanResource ;
                        rdfs:comment "Personnel devoted to technical suport."@en ;
                        rdfs:label "Personal tècnic"@ca-ad ,
                                   "Personal tècnic"@ca-es ,
                                   "Personal técnico"@es ,
                                   "Personnel technique"@fr ,
                                   "Pessoal técnico"@pt ,
                                   12,
                                   "Technical personnel"@en .

"""

## Synchronization algorithms

### Graph diff

In [4]:
from rdflib.compare import graph_diff, to_isomorphic
from rdflib.graph import Graph


source_g = Graph().parse(format='turtle', data=source_content)
target_g = Graph().parse(format='turtle', data=target_content)

source_g_iso = to_isomorphic(source_g)
target_g_iso = to_isomorphic(target_g)

in_both, in_first, in_second = graph_diff(source_g_iso, target_g_iso)

In [5]:
def print_triples(graph):
    for triple in graph:
        print(triple)
        print('-' * 20)
    print('\n')

print("Triples in both: ")
print_triples(in_both)

print("Removed triples: ")
print_triples(in_first)

print("Added triples: ")
print_triples(in_second)

Triples in both: 
(rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#AdministrativePersonnel'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
--------------------
(rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#AdministrativePersonnel'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#HumanResource'))
--------------------
(rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#HumanResource'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
--------------------


Removed triples: 
(rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#ResearchPersonne

In [13]:
import rdflib

for triple in in_second:
    if isinstance(triple[2], rdflib.term.Literal):
        print(triple[2].datatype)

http://www.w3.org/2001/XMLSchema#integer
None
None
None
None
None
None
None


## Using hercules_sync

In [5]:
from hercules_sync.git import GitFile
from hercules_sync.synchronization import GraphDiffSyncAlgorithm, OntologySynchronizer

gitfile = GitFile(None, source_content, target_content)
algorithm = GraphDiffSyncAlgorithm()
synchronizer = OntologySynchronizer(algorithm)

In [6]:
ops = algorithm.do_algorithm(gitfile)

In [7]:
len(ops)

17

In [8]:
str(ops[1])

'AdditionOperation: http://www.asio.es/asioontologies/asio#TechnicalPersonnel                  http://www.w3.org/2000/01/rdf-schema#label                  Personal tècnic'

In [9]:
username = "WikibaseAdmin"
password = "WikibaseDockerAdminPass"
mediawiki_api_url='http://156.35.94.149:8181/w/api.php'
sparql_endpoint_url='http://156.35.94.149:8282/proxy/wdqs/bigdata/namespace/wdq/sparql'

In [10]:
from hercules_sync.triplestore import WikibaseAdapter

adapter = WikibaseAdapter(mediawiki_api_url, sparql_endpoint_url, username, password)

http://156.35.94.149:8181/w/api.php
Successfully logged in as WikibaseAdmin


In [11]:
str(ops[1])

'AdditionOperation: http://www.asio.es/asioontologies/asio#TechnicalPersonnel                  http://www.w3.org/2000/01/rdf-schema#label                  Personal tècnic'

In [12]:
ops[0].execute(adapter)

{rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): 'P1', rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#ResearchPersonnel'): 'Q1', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'): 'Q2', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'): 'P2', rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#HumanResource'): 'Q3', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#disjointWith'): 'P3', rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#AdministrativePersonnel'): 'Q4', rdflib.term.URIRef('http://www.semanticweb.org/spitxa/ontologies/2020/1/asio-human-resource#ChangedPersonnel'): 'Q5', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): 'P4', rdflib.term.URIRef('http://www.asio.es/asioontologies/asio#TechnicalPersonnel'): 'Q6', rdflib.term.Literal('12', datatype=rdflib.term.URIRef('http://www.w3.org

/Users/laika/Documents/envs/herc_sync/lib/python3.6/site-packages/wikidataintegrator/wdi_core.py:219: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")
/Users/laika/Documents/envs/herc_sync/lib/python3.6/site-packages/wikidataintegrator/wdi_core.py:175: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")
